In [1]:
import mne
import pandas as pd
import numpy as np

### Reading In the Data

In [2]:
%%capture
raw = mne.io.read_raw_gdf(r'.\BCICIV_2a_gdf\A01T.gdf', preload=True)

In [3]:
channel_names = raw.info['ch_names']

print(channel_names)

['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16', 'EOG-left', 'EOG-central', 'EOG-right']


In [4]:
len(channel_names)

25

In [5]:
channels_to_keep = ['EEG-1', 'EEG-3', 'EEG-C3', 'EEG-C4', 'EEG-14', 'EEG-Pz', 'EEG-15']     # F3, F4, C3, C4, Pz, P3, P4

In [6]:
ordered_event_id = {
    '276'  : 1,  # eyes open, idling
    '277'  : 2,
    '768'  : 3,
    '769'  : 4, # left/right cue
    '770'  : 5, # left/right cue
    '771'  : 6,
    '772'  : 7, 
    '783'  : 8,
    '1023' : 9,
    '1072' : 10,
    '32766': 11
}

### Filtering and Cleaning the Data

In [7]:
filtered_raw = raw.copy().pick(channels_to_keep).filter(1, 30, picks=channels_to_keep)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s finished


In [8]:
events, event_id = mne.events_from_annotations(filtered_raw, event_id = ordered_event_id)
events[:10]     # format (sample_index, previous_event_value, event_id)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


array([[    0,     0,    11],
       [    0,     0,     1],
       [29683,     0,    11],
       [29683,     0,     2],
       [49955,     0,    11],
       [49955,     0,    10],
       [91518,     0,    11],
       [91868,     0,     3],
       [92368,     0,     7],
       [93871,     0,     3]])

### Isolating the events that we want to keep--events 1, 4, and 5

In [9]:
indices = []
for i in range(len(events) - 1):
    if events[i][2] == 1 or events[i][2] == 4 or events[i][2] == 5: # isolating our desired labels
        start = events[i][0]
        end = events[i + 1][0]
        if end - start > 0:
            indices.append((int(start), int(end), int(events[i][2]))) # storing duration + event labels

In [10]:
indices[:10]

[(0, 29683, 1),
 (96289, 97741, 5),
 (98241, 99749, 4),
 (100249, 101860, 4),
 (102360, 103777, 5),
 (108258, 109756, 5),
 (112162, 113557, 4),
 (114057, 115529, 4),
 (116029, 117473, 4),
 (119940, 121560, 5)]

##### Slicing these events from the original data

In [11]:
filtered_data = filtered_raw.get_data().T
filtered_data.shape

(672528, 7)

In [12]:
indices[0]

(0, 29683, 1)

In [13]:
filtered_data[indices[0][0]:indices[0][1], :].shape

(29683, 7)

In [14]:
filtered_data.shape

(672528, 7)

In [15]:
slices = []
for ind in indices:
    package = (filtered_data[ind[0]:ind[1], : ], ind[2]) # (information slice, label)
    slices.append(package)

In [20]:
len(slices)

144

In [28]:
slices[0][1]

1

In [27]:
slices[0][0].shape

(29683, 7)

In [30]:
slices[1][1]

5

In [32]:
slices[1][0].shape

(1452, 7)

In [35]:
slices[2][1]

4

In [34]:
slices[2][0].shape

(1508, 7)

In [17]:
# del event_id['32766']

In [18]:
# filtering the data and getting the events
# epochs = mne.Epochs(filtered_raw, events_modified, event_id = event_id, tmin = -0.5, tmax = 1.0, baseline = (None, 0), preload=True)

note for myself
- can try and isolate the events into blocks that we can then classify
- need to filter the data before isolating these events
- mne.events_from_annotations --> [time...? (not exactly the time but the number of samples preceding it I think, we can ignore this column, event id]

filtering notes
- maybe 1-30 Hz would be a good range for our data

picking events notes
- can try and plot this data to make sure what I have is properly segmented..?